In [1]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt

import aot


In [2]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)
total_video_number = core_settings["various"]["total_video_number"]
settings_root_path = base_dir / core_settings["paths"]["settings_path"]
main_settings_path = settings_root_path / "main"

video_names = []
for i in range(1, 1 + total_video_number):
    # 0 padding for video name
    video_name = str(i).zfill(4) + ".mp4"
    video_names.append(video_name)
print("videos number: ", len(video_names))

origin_videos_number = len(video_names)
subject_number = core_settings["various"]["subject_number"]
session_number = core_settings["various"]["session_number"]
run_number = core_settings["various"]["run_number"]
# only for main run
blank_number_per_run = core_settings["various"]["blank_number_per_run"]
unique_video_number_per_session = core_settings["various"][
    "unique_video_number_per_session"
]
video_number_per_run_main = int((unique_video_number_per_session * 4) / run_number)
# only 1 subject and 1 session for grading
video_number_per_run_grading = int(origin_videos_number / run_number)

videos number:  2214


In [3]:
def read_settings_file(subject, ses, run):
    settings_file = main_settings_path / f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(ses).zfill(2)}_run_{str(run).zfill(2)}.yml"
    settings = yaml.load(open(settings_file), Loader=yaml.FullLoader)
    return settings

In [4]:
def read_settings_files_session(subject, ses):#return a list of settings files for a given session
    settings_files = [read_settings_file(subject, ses, run) for run in range(1,run_number+1)]
    sessions_movies = [settings['stimuli']['movie_files'] for settings in settings_files]
    return sessions_movies

In [5]:
def check_video_replay_of_session(session_movies):
    #check if the same video is replayed in a session
    #append all the lists of movies in a session
    all_movies = []
    for run_movies in session_movies:
        all_movies += run_movies
    #remove the blanks
    all_movies = [movie for movie in all_movies if movie != 'blank']
    #check if there is a replay
    stack = []
    for movie in all_movies:
        if movie in stack:
            stack.remove(movie)
        else:
            stack.append(movie)
    return len(stack) == 0

def check_video_replay_all_session():
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            session_movies = read_settings_files_session(subject, ses)
            #print(f"subject {subject}, session {ses} movies: {session_movies}")
            is_replay = check_video_replay_of_session(session_movies)
            print(f"subject {subject}, session {ses} replay: {is_replay}")

check_video_replay_all_session()

subject 1, session 1 replay: True
subject 1, session 2 replay: True
subject 1, session 3 replay: True
subject 1, session 4 replay: True
subject 1, session 5 replay: True
subject 1, session 6 replay: True
subject 1, session 7 replay: True
subject 1, session 8 replay: True
subject 2, session 1 replay: True
subject 2, session 2 replay: True
subject 2, session 3 replay: True
subject 2, session 4 replay: True
subject 2, session 5 replay: True
subject 2, session 6 replay: True
subject 2, session 7 replay: True
subject 2, session 8 replay: True
subject 3, session 1 replay: True
subject 3, session 2 replay: True
subject 3, session 3 replay: True
subject 3, session 4 replay: True
subject 3, session 5 replay: True
subject 3, session 6 replay: True
subject 3, session 7 replay: True
subject 3, session 8 replay: True
subject 4, session 1 replay: True
subject 4, session 2 replay: True
subject 4, session 3 replay: True
subject 4, session 4 replay: True
subject 4, session 5 replay: True
subject 4, ses

In [6]:
def check_video_direction_duality_of_session(session_movies):
    #check whether the videos in a session are duality pairs(R_S_movie,S_movie)
    #append all the lists of movies in a session
    def check_duality(video1,video2):
        '''
        if video1.startswith('R_') and video2.startswith('S_'):
            return video1[4:] == video2[2:]
        elif video1.startswith('S_') and video2.startswith('R_'):
            return video1[2:] == video2[4:]
        else:
            return False
        '''
        if video1.endswith('_fw.mp4') and video2.endswith('_rv.mp4'):
            return video1[:-7] == video2[:-7]
        elif video1.endswith('_rv.mp4') and video2.endswith('_fw.mp4'):
            return video1[:-7] == video2[:-7]
        else:
            return False
    def is_duality_in_stack(video, video_stack):#with side effect that remove the duality pair
        for v in video_stack:
            if check_duality(video,v):
                video_stack.remove(v)
                return True
        return False
    all_movies = []
    for run_movies in session_movies: 
        all_movies += run_movies
    #remove the blanks
    all_movies = [movie for movie in all_movies if movie != 'blank']
    stack = []
    for movie in all_movies:
        if not is_duality_in_stack(movie, stack):
            stack.append(movie)
    return len(stack) == 0

def check_video_direction_duality_all_session():
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            session_movies = read_settings_files_session(subject, ses)
            #print(f"subject {subject}, session {ses} movies: {session_movies}")
            is_duality = check_video_direction_duality_of_session(session_movies)
            print(f"subject {subject}, session {ses} duality: {is_duality}")

check_video_direction_duality_all_session()
    

subject 1, session 1 duality: True
subject 1, session 2 duality: True
subject 1, session 3 duality: True
subject 1, session 4 duality: True
subject 1, session 5 duality: True
subject 1, session 6 duality: True
subject 1, session 7 duality: True
subject 1, session 8 duality: True
subject 2, session 1 duality: True
subject 2, session 2 duality: True
subject 2, session 3 duality: True
subject 2, session 4 duality: True
subject 2, session 5 duality: True
subject 2, session 6 duality: True
subject 2, session 7 duality: True
subject 2, session 8 duality: True
subject 3, session 1 duality: True
subject 3, session 2 duality: True
subject 3, session 3 duality: True
subject 3, session 4 duality: True
subject 3, session 5 duality: True
subject 3, session 6 duality: True
subject 3, session 7 duality: True
subject 3, session 8 duality: True
subject 4, session 1 duality: True
subject 4, session 2 duality: True
subject 4, session 3 duality: True
subject 4, session 4 duality: True
subject 4, session 5

In [7]:
def count_same_video_in_one_run(subject, ses, run):
    #count the number of same video in one run
    settings = read_settings_file(subject, ses, run)
    movies = settings['stimuli']['movie_files']
    movies = [movie for movie in movies if movie != 'blank']
    count = 0
    stack = []
    for movie in movies:
        if movie in stack:
            stack.remove(movie)
            count += 1
        else:
            stack.append(movie)
    return count

def count_same_video_in_all_run():
    count_list = []
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            for run in range(1,run_number+1):
                count = count_same_video_in_one_run(subject, ses, run)
                count_list.append(count)
                print(f"subject {subject}, session {ses} ,run {run} same video count: {count}")
    return count_list

count_same_video_in_all_run()


subject 1, session 1 ,run 1 same video count: 2
subject 1, session 1 ,run 2 same video count: 1
subject 1, session 1 ,run 3 same video count: 4
subject 1, session 1 ,run 4 same video count: 4
subject 1, session 1 ,run 5 same video count: 7
subject 1, session 1 ,run 6 same video count: 6
subject 1, session 1 ,run 7 same video count: 2
subject 1, session 1 ,run 8 same video count: 4
subject 1, session 1 ,run 9 same video count: 1
subject 1, session 1 ,run 10 same video count: 0
subject 1, session 2 ,run 1 same video count: 5
subject 1, session 2 ,run 2 same video count: 1
subject 1, session 2 ,run 3 same video count: 5
subject 1, session 2 ,run 4 same video count: 4
subject 1, session 2 ,run 5 same video count: 1
subject 1, session 2 ,run 6 same video count: 5
subject 1, session 2 ,run 7 same video count: 3
subject 1, session 2 ,run 8 same video count: 6
subject 1, session 2 ,run 9 same video count: 6
subject 1, session 2 ,run 10 same video count: 4
subject 1, session 3 ,run 1 same video

[2,
 1,
 4,
 4,
 7,
 6,
 2,
 4,
 1,
 0,
 5,
 1,
 5,
 4,
 1,
 5,
 3,
 6,
 6,
 4,
 2,
 1,
 2,
 4,
 4,
 4,
 7,
 4,
 2,
 5,
 2,
 3,
 3,
 3,
 4,
 1,
 7,
 4,
 6,
 5,
 3,
 3,
 6,
 8,
 6,
 5,
 6,
 7,
 5,
 4,
 2,
 5,
 4,
 2,
 6,
 4,
 7,
 6,
 7,
 5,
 3,
 7,
 7,
 5,
 4,
 5,
 4,
 2,
 3,
 3,
 3,
 4,
 3,
 4,
 2,
 4,
 3,
 3,
 6,
 2,
 2,
 3,
 3,
 2,
 4,
 2,
 2,
 3,
 4,
 5,
 5,
 4,
 4,
 2,
 3,
 6,
 3,
 4,
 5,
 0,
 2,
 5,
 7,
 4,
 3,
 5,
 3,
 3,
 4,
 3,
 6,
 9,
 4,
 8,
 3,
 3,
 4,
 5,
 4,
 4,
 2,
 4,
 0,
 4,
 4,
 8,
 3,
 2,
 6,
 4,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 8,
 2,
 2,
 5,
 7,
 1,
 4,
 3,
 6,
 2,
 2,
 2,
 3,
 4,
 2,
 6,
 2,
 3,
 4,
 2,
 2,
 4,
 1,
 7,
 3,
 1,
 0,
 4,
 3,
 4,
 4,
 5,
 7,
 3,
 6,
 4,
 3,
 5,
 4,
 4,
 4,
 3,
 4,
 2,
 1,
 7,
 2,
 4,
 4,
 3,
 5,
 6,
 4,
 3,
 3,
 3,
 5,
 8,
 2,
 2,
 4,
 3,
 3,
 0,
 3,
 7,
 5,
 5,
 6,
 6,
 1,
 4,
 2,
 0,
 2,
 5,
 4,
 3,
 5,
 7,
 2,
 4,
 4,
 4,
 2,
 1,
 7,
 3,
 2,
 3,
 2,
 2,
 4,
 5,
 3,
 6,
 4,
 7,
 4,
 2,
 5,
 2,
 5,
 2,
 4,
 1,
 4,
 2,
 1,
 3,
 3,


In [8]:
def check_uniqe_video_number_bothdirection(sub, ses):
    sessions_movies = read_settings_files_session(sub, ses)
    video_names = []
    for movies in sessions_movies:
        for movie in movies:
            video_names.append(movie.split("/")[-1].split(".")[0])
    videoset = set(video_names)
    print(videoset)
    print("unique video number: ", len(videoset))
    return videoset

'''
def check_uniqe_video_number_uniqe_original(sub, ses):
    def deR(name):
        return name[2:]

    sessions_movies = read_settings_files_session(sub, ses)
    video_names = []
    for movies in sessions_movies:
        for movie in movies:
            video_names.append(movie.split("/")[-1].split(".")[0])
    videoset = set(video_names)
    new_video_list = []
    for video in videoset:
        if video[0] == "R":
            new_video_list.append(deR(video))
        else:
            new_video_list.append(video)

    videoset = set(new_video_list)
    # print(videoset)
    print(
        "unique video number sub {sub} ses {ses}: ".format(sub=sub, ses=ses),
        len(videoset),
    )

    return videoset
    '''

In [9]:
for sub in range(1, subject_number + 1):
    for ses in range(1, session_number + 1):
        check_uniqe_video_number_bothdirection(sub, ses)

unique video number sub 1 ses 1:  361
unique video number sub 1 ses 2:  361
unique video number sub 1 ses 3:  361
unique video number sub 1 ses 4:  361
unique video number sub 1 ses 5:  361
unique video number sub 1 ses 6:  361
unique video number sub 1 ses 7:  361
unique video number sub 1 ses 8:  361
unique video number sub 2 ses 1:  361
unique video number sub 2 ses 2:  361
unique video number sub 2 ses 3:  361
unique video number sub 2 ses 4:  361
unique video number sub 2 ses 5:  361
unique video number sub 2 ses 6:  361
unique video number sub 2 ses 7:  361
unique video number sub 2 ses 8:  361
unique video number sub 3 ses 1:  361
unique video number sub 3 ses 2:  361
unique video number sub 3 ses 3:  361
unique video number sub 3 ses 4:  361
unique video number sub 3 ses 5:  361
unique video number sub 3 ses 6:  361
unique video number sub 3 ses 7:  361
unique video number sub 3 ses 8:  361
unique video number sub 4 ses 1:  361
unique video number sub 4 ses 2:  361
unique video